Загрузка и предобработка данных CIFAR-10

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np

# Загрузка данных CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Нормализация данных (приведение значений пикселей к диапазону [0, 1])
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Преобразование меток в one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Вывод информации о данных
print("Размер обучающей выборки:", x_train.shape)
print("Размер тестовой выборки:", x_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 67s 0us/step
Размер обучающей выборки: (50000, 32, 32, 3)
Размер тестовой выборки: (10000, 32, 32, 3)


Классификация изображений с помощью SVM и KNN

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Используем подвыборку данных для ускорения обучения
x_train_subset = x_train[:5000].reshape(5000, -1)  # Преобразуем изображения в одномерные векторы
y_train_subset = np.argmax(y_train[:5000], axis=1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train_subset, y_train_subset)
y_pred_knn = knn.predict(x_test_flat)
accuracy_knn = accuracy_score(np.argmax(y_test, axis=1), y_pred_knn)
print(f"Точность KNN: {accuracy_knn * 100:.2f}%")

# SVM
svm = SVC()
svm.fit(x_train_subset, y_train_subset)
y_pred_svm = svm.predict(x_test_flat)
accuracy_svm = accuracy_score(np.argmax(y_test, axis=1), y_pred_svm)
print(f"Точность SVM: {accuracy_svm * 100:.2f}%")

Точность KNN: 25.90%
Точность SVM: 44.34%


Создание многослойного перцептрона (MLP)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, InputLayer

# Создание модели MLP
model_mlp = Sequential([
    InputLayer(shape=(32, 32, 3)),  # Указываем входную форму
    Flatten(),  # Преобразуем изображение в одномерный вектор
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')  # Выходной слой для 10 классов
])

# Компиляция модели
model_mlp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
history_mlp = model_mlp.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Оценка точности на тестовых данных
test_loss_mlp, test_accuracy_mlp = model_mlp.evaluate(x_test, y_test)
print(f"Точность MLP на тестовых данных:  {test_accuracy_mlp * 100:.2f}%")

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 33s 49ms/step - accuracy: 0.2225 - loss: 2.2143 - val_accuracy: 0.3444 - val_loss: 1.8458
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 25s 40ms/step - accuracy: 0.3221 - loss: 1.8649 - val_accuracy: 0.3698 - val_loss: 1.7639
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.3489 - loss: 1.7929 - val_accuracy: 0.3838 - val_loss: 1.7284
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 40s 41ms/step - accuracy: 0.3598 - loss: 1.7747 - val_accuracy: 0.3781 - val_loss: 1.7266
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 36s 58ms/step - accuracy: 0.3718 - loss: 1.7304 - val_accuracy: 0.3848 - val_loss: 1.6899
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 49ms/step - accuracy: 0.3824 - loss: 1.7176 - val_accuracy: 0.3992 - val_loss: 1.6707
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.3893 - loss: 1.6963 - val_accuracy: 0.4216 - val_loss: 1.6306
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.3996 - loss: 1.6642 - 

Создание сверточной нейронной сети (CNN)

In [7]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# Создание модели CNN
model_cnn = Sequential([
    InputLayer(shape=(32, 32, 3)),  # Указываем входную форму
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

# Компиляция модели
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
history_cnn = model_cnn.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Оценка точности на тестовых данных
test_loss_cnn, test_accuracy_cnn = model_cnn.evaluate(x_test, y_test)
print(f"Точность CNN на тестовых данных: {test_accuracy_cnn * 100:.2f}%")

c:\Users\YasperMoglot\Desktop\Ycheba\4course\raspoznavanie-obrazov\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 43ms/step - accuracy: 0.3279 - loss: 1.8228 - val_accuracy: 0.5073 - val_loss: 1.3832
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.5255 - loss: 1.3230 - val_accuracy: 0.5931 - val_loss: 1.1542
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 40s 45ms/step - accuracy: 0.5971 - loss: 1.1449 - val_accuracy: 0.6218 - val_loss: 1.0782
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.6331 - loss: 1.0495 - val_accuracy: 0.6462 - val_loss: 1.0005
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 40s 43ms/step - accuracy: 0.6607 - loss: 0.9575 - val_accuracy: 0.6611 - val_loss: 0.9697
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.6871 - loss: 0.8948 - val_accuracy: 0.6855 - val_loss: 0.9086
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.7118 - loss: 0.8281 - val_accuracy: 0.6850 - val_loss: 0.8995
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 47ms/step - accuracy: 0.7234 - loss: 0.7842 - 

Визуализация результатов

In [ ]:
import matplotlib.pyplot as plt

# Функция для отображения изображений и предсказаний
def plot_images(images, labels, predictions, num_images=5):
    plt.figure(figsize=(15, 5))
    for i in range(num_images):
        plt.subplot(1, num_images, i+1)
        plt.imshow(images[i])
        plt.title(f"True: {np.argmax(labels[i])}\nPred: {predictions[i]}")
        plt.axis('off')
    plt.show()

# Предсказания для тестовых данных
y_pred_mlp = np.argmax(model_mlp.predict(x_test), axis=1)
y_pred_cnn = np.argmax(model_cnn.predict(x_test), axis=1)

# Визуализация результатов
plot_images(x_test, y_test, y_pred_knn, num_images=5)  # KNN
plot_images(x_test, y_test, y_pred_svm, num_images=5)  # SVM
plot_images(x_test, y_test, y_pred_mlp, num_images=5)  # MLP
plot_images(x_test, y_test, y_pred_cnn, num_images=5)  # CNN